In [2]:
########################################################
###  Small sledgehammer to calculate bond lenghts,   ###
###  coordination numbers, Hydrogen Bond lenghts     ###
###  averages etc. Currently only suitable for       ###
###  orthogonal and hexagonal systems. It is also    ###
###  only suitable for VASP files                    ###
###                                                  ###
###  Author : Adam Symington                         ###
###  Contact : ars44@bath.ac.uk                      ###
###                                                  ###
###  Usage : When prompted enter the species you     ###
###          want to calculate bond lenghts for      ###
###          e.g Gd (Case sensitive)                 ###
###          You will then be prompted for the       ###
###          other species e.g. O                    ###
###          Finally you will be asked for the max   ###
###          bond lenght to be considered e.g. 3.0   ###
###                                                  ###
###  It does not automatically work out the cell     ###
###  dimensions at present so the angles will need   ###
###  to be specified.                                ###
########################################################

## Import Modules ##
import numpy as np
import math as math
from math import pi
pi = math.pi
f = open('CONTCAR','r')

## read in structure name ##
line = f.readline().split()
name = ''
for i in range(len(line)):
    name = name + line[i] + ' '

## read in scale factor  ##
scale = str(f.readline().split()[0])

## read in unit cell lattice vectors ##
lat = np.zeros((3,3))
for i in range(3):
    line = f.readline().split()
    for j in range(3):
        lat[i,j]=float(line[j])
x = lat[0,0], lat[0,1], lat[0,2]
x = np.asarray(x)
y = lat[1,0], lat[1,1], lat[1,2]
y = np.asarray(y)
z = lat[2,0], lat[2,1], lat[2,2]
z = np.asarray(z)

##########   This Function Calculates  and returns the lenght of an edge
def get_lenghts(A):
    if A[0] == 0.0 and A[1] == 0.0:
        lenght = A[2]
    elif A[0] == 0.0 and A[2] == 0.0:
        lenght = A[1]
    elif A[1] == 0.0 and A[2] == 0.0:
        lenght = A[0]
    elif A[0] != 0.0 and A[1] != 0.0:
        lenght = math.sqrt((A[0] ** 2) + (A[1] ** 2))
    elif A[1] != 0.0 and A[2] != 0.0:
        lenght = math.sqrt((A[1] ** 2) + (A[2] ** 2))
    elif A[0] != 0.0 and A[2] != 0.0:
        lenght = math.sqrt((A[0] ** 2) + (A[2] ** 2))
    elif A[0] != 0.0 and A[1] != 0.0 and A[2] != 0.0:
        lenght = math.sqrt((A[0] ** 2) + (A[1] ** 2) + (A[2] ** 2))
    return(lenght)
        
x_vec = get_lenghts(x)
y_vec = get_lenghts(y)
z_vec = get_lenghts(z)
alpha = 90
beta = 90
gamma = 60
Coord_system = "Hexagonal" 

## Determine a transformation matrix for the conversion of Direct to Cartesian ####
def transformation(alpha, beta, gamma, x_vec, y_vec, z_vec):    
    alpha = np.deg2rad(alpha)
    beta = np.deg2rad(beta)
    gamma = np.deg2rad(gamma)
    cosa = np.cos(alpha)
    sina = np.sin(alpha)
    cosb = np.cos(beta)
    sinb = np.sin(beta)
    cosg = np.cos(gamma)
    sing = np.sin(gamma)
    volume = 1.0 - cosa**2.0 - cosb**2.0 - cosg**2.0 + 2.0 * cosa * cosb * cosg
    volume = np.sqrt(volume)
    r = np.zeros((3, 3))
    r[0, 0] = x_vec
    r[0, 1] = y_vec * cosg
    r[0, 2] = z_vec * cosb
    r[1, 1] = y_vec * sing
    r[1, 2] = z_vec * (cosa - cosb * cosg) / sing
    r[2, 2] = z_vec * volume / sing
    return r

r = transformation(alpha, beta, gamma, x_vec, y_vec, z_vec)

#### Calculate X and Y scalers ###########
y_scale = y_vec * (math.sin(np.deg2rad(gamma)))
x_scale = (math.sqrt((y_vec ** 2) - (y_scale ** 2)))
    
## read in number of atom types, number of atoms of each type ##
atom_types = f.readline().split()
atom_nums = f.readline().split()
atom_nums = list(map(int, atom_nums))
total_atoms = sum(atom_nums)

#read in atom coordinate convention
convention = f.readline().split()[0]

#read in atomic positions
Coord = np.zeros((total_atoms,3))
for i in range(total_atoms):
    line = f.readline().split()
    for j in range(3):
        Coord[i,j] = float(line[j])

## Convert from Fractional to Cartesian
if convention == "Direct":
    Coord_Cart = ([])
    for row in Coord:
        Coord = np.dot(r, row)
        Coord_Cart = np.append(Coord_Cart, Coord)
    Coord_Cart = np.reshape(Coord_Cart, (total_atoms, 3))
elif convention != "Direct":
    Coord_Cart = Coord

x_coords = Coord_Cart[:,[0]]
y_coords = Coord_Cart[:,[1]]
z_coords = Coord_Cart[:,[2]]


## Declare what bonds are to be counted ##
Bond_check = input("Enter the atom you want to calculate bond lenghts for: ")
Bond_check = str(Bond_check)
Coord_spec = input("Enter the coordinating species: ")
Coord_spec = str(Coord_spec)
Max_lenght = input("Enter the Maximum bond lenght to be considered: ")
Max_lenght = float(Max_lenght)


## Read Species at center of CN sphere
def get_x_M(X):
    A = np.array([])
    B = np.array([])
    C = np.array([])
    for i, j in enumerate(atom_types):
        if j == X:
            lower_lim = sum(atom_nums[:i])
            n_M = (atom_nums[i])
            upper_lim = lower_lim + n_M
            ran = upper_lim - lower_lim
            Top_M_x_cart = x_coords[lower_lim:upper_lim]
            Top_M_y_cart = y_coords[lower_lim:upper_lim]
            Top_M_z_cart = z_coords[lower_lim:upper_lim]
            A = np.append(A, Top_M_x_cart)
            B = np.append(B, Top_M_y_cart)
            C = np.append(C, Top_M_z_cart)
    return A, B, C


## Calculate difference in X, Y and Z coordinates for two species
def differences(Ax, Ay, Az, Bx, By, Bz):
    ABx = Ax - Bx
    ABy = Ay - By
    ABz = Az - Bz
    return ABx, ABy, ABz

## PBC for Orthog systems
def PBC_orthogonal(A, B, C):
    
    if (A) > (x_vec*0.5):
        A = A - x_vec 
    elif (-A) > (x_vec*0.5):
        A = A + x_vec
             
    if (B) > (y_vec*0.5):
        B = B - y_vec
    elif (-B) > (y_vec*0.5):
        B = B + y_vec
            
    if (C) > (z_vec*0.5):
        C = C - z_vec
    elif (-C) > (z_vec*0.5):
        C = C + z_vec
    return A, B, C

## PBC for Hexagonal systems
def PBC_hexagonal(A, B, C):    
  
    if A > (x_vec * 0.5) and B > (y_vec * 0.5):
        B = B - y_scale
        A = A - x_scale
    elif -A > (x_vec * 0.5) and -B > (y_vec * 0.5):
        B = B + y_scale
        A = A + x_scale
        
    if A < (x_vec * 0.5) and B > (y_vec * 0.5):
        B = B - y_scale
        A = A - x_scale
    elif -A < (x_vec * 0.5) and -B > (y_vec * 0.5):
        B = B + y_scale
        A = A + x_scale
      
    if (A) > (x_vec * 0.5):
        A = A - x_vec
    elif -(A) > (x_vec * 0.5):
        A = A + x_vec
        
    if C > (z_vec * 0.5):
        C = C - z_vec
    elif (-C) > (z_vec * 0.5):
        C = C + z_vec 
    return A, B, C

## Calculate Bond lenghts between two species
def get_bond_lenght (A, B, C):       
    Bond_lenght = math.sqrt((A ** 2) + (B ** 2) + (C ** 2)) 
    return(Bond_lenght)

# Calculate O-H-O angle
def get_angle(A, B, C, AB, CB):
    OHOangle = (A + B + C)
    OHOangle = (OHOangle/(AB * CB))
    OHOangle = np.around(OHOangle, decimals=8)
    OHOangle = float(OHOangle)
    OHOangle = math.acos(OHOangle)
    OHOangle = OHOangle*(180/pi)
    return OHOangle

#def find_water():

    
#def find_OH():
    
    
    
M_x, M_y, M_z = get_x_M(Bond_check)
spec_x, spec_y, spec_z  = get_x_M(Coord_spec)
H_x, H_y, H_z = get_x_M("H")
O_x, O_y, O_z = get_x_M("O")

n_M = np.size(M_x)
n_X = np.size(spec_x)
n_H = np.size(H_x)
n_O = np.size(O_x)

##################################################################
#########         Find Water and OH groups        ###############
#################################################################
HOHx = np.array([])
HOHy = np.array([])
HOHz = np.array([])
OHx = np.array([])
OHy = np.array([])
OHz = np.array([])

for i in range(0, n_O):
    count = 0
    for j in range(0, n_H):
        ABx, ABy, ABz = differences(O_x[i], O_y[i], O_z[i], H_x[j], H_y[j], H_z[j])
        if Coord_system == "Hexagonal":
            ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        elif Coord_system != "Hexagonal":
            ABx, ABy, ABz = PBC_orthogonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        if Bond_lenght < 1.05:
            count = count+1
    if count == 1:
        OHx = np.append(OHx, O_x[i])
        OHy = np.append(OHy, O_y[i])
        OHz = np.append(OHz, O_z[i])
    elif count == 2:
        HOHx = np.append(HOHx, O_x[i])
        HOHy = np.append(HOHy, O_y[i])
        HOHz = np.append(HOHz, O_z[i])
        
n_OH = np.size(OHx)
n_HOH = np.size(HOHx)
Bond_lenght_array = np.array([])

print()
print("##############################################################")
print("             Bond Lenght Information for", Bond_check, "-", Coord_spec    )
print("##############################################################")
print()
for i in range(0, n_M):
    M_array = np.array([])
    for j in range(0, n_X):
        ABx, ABy, ABz = differences(spec_x[j], spec_y[j], spec_z[j], M_x[i], M_y[i], M_z[i]) 
        if Coord_system == "Hexagonal":
            ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        elif Coord_system != "Hexagonal":
            ABx, ABy, ABz = PBC_orthogonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        if Bond_lenght < 3.0:
            M_array = np.append(M_array, Bond_lenght)
    if M_array.size:
        M_array = np.around(M_array, decimals=3)
        print()
        print(Bond_check, "-", i + 1, (np.around(M_array, decimals=3)))
        print()
        print("Average Bond Lenght for ", Bond_check, "-", i + 1, (np.around(np.average(M_array), decimals=3)))
        print("Range in Bond Lenght:", np.amax(np.around(M_array, decimals=3)), " - ", np.amin(np.around(M_array, decimals=3)))
        print()
        print()
        print("Coordination Number - ", M_array.size)
        print()
    elif not M_array.size:
        print(Bond_check, "-", i + 1)
        print("There are no", Coord_spec, "within", Max_lenght, "of", Bond_check, "-", i+1)
        print("Try increasing the Bond lenght parameter")
    Bond_lenght_array = np.append(Bond_lenght_array, M_array)  
    
print()
print("Average", Bond_check, "-", Coord_spec, "Bond Lenght: ", np.average(np.around(Bond_lenght_array, decimals=3)))
print("Range in Bond Length Values: ", np.amax(np.around(Bond_lenght_array, decimals=3)), " - ", np.amin(np.around(Bond_lenght_array, decimals=3)))
print()
print("Standard Deviation: ", np.std(np.around(Bond_lenght_array, decimals=3)))

print()
print("##############################################################")
print("                 Water Adsorption information                 ")
print("##############################################################")
print()
print("Total water molecules considered: ", (n_H/2))
print("Total number of Dissociative water in system:", (n_OH/2))
print("                         Number of OH groups:", n_OH)
print()
print("Total number of Associative water in system: ", n_HOH)
print() 

print("##############################################################")
print("      The bonds between ", Bond_check, "and OH in dissociative water")
print("##############################################################")
print()
MOH_tot = np.array([])
for i in range(0, n_M):
    MOH_array = np.array([])
    for j in range(0, n_OH):
        ABx, ABy, ABz = differences(OHx[j], OHy[j], OHz[j], M_x[i], M_y[i], M_z[i]) 
        if Coord_system == "Hexagonal":
            ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        elif Coord_system != "Hexagonal":
            ABx, ABy, ABz = PBC_orthogonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        if Bond_lenght < 3.0: 
            MOH_array = np.append(MOH_array, Bond_lenght)
    if MOH_array.size:
        print(Bond_check, "-", i+1 ,"- OH", (np.around(MOH_array, decimals=3)))
        MOH_tot = np.append(MOH_tot, MOH_array)
    elif not MOH_array.size:
        print(Bond_check, "-", i+1, "-", "There are no ", Bond_check, "- OH bonds")
print()
if MOH_tot.size:
    print("Average ", Bond_check, "- OH", (np.around((np.average(MOH_tot)), decimals=3)))
print()

print("##############################################################")
print("      The bonds between ", Bond_check, "and HOH in Associative water")
print("##############################################################")
print() 

MOHH_tot = np.array([])
for i in range(0,n_M):
    MHOH_array = np.array([])
    for j in range(0, n_HOH):
        ABx, ABy, ABz = differences(HOHx[j], HOHy[j], HOHz[j], M_x[i], M_y[i], M_z[i]) 
        if Coord_system == "Hexagonal":
            ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        elif Coord_system != "Hexagonal":
            ABx, ABy, ABz = PBC_orthogonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        if Bond_lenght < 3.0: 
            MHOH_array = np.append(MHOH_array, Bond_lenght)
    if MHOH_array.size:
        print(Bond_check, "-", i+1 ,"- OHH", (np.around(MHOH_array, decimals=3)))
        MOHH_tot = np.append(MOHH_tot, MHOH_array)
    elif not MHOH_array.size:
        print(Bond_check, "-", i+1, "-", "There are no ", Bond_check, "- OH bonds")
print()
if MOHH_tot.size:
    print("Average ", Bond_check, "- OHH", (np.around((np.average(MOHH_tot)), decimals=3)))
print()

print("##############################################################")
print("                   Hydrogen Bond Section                      ")
print("##############################################################")
print()
print("Definition of Hydrogen Bonding Strength")
print("Strong H-Bond = 1.2- 1.6")
print("Medium H-Bond = 1.6 - 2.2")
print("Weak H-Bond = 2.2 - 3.0")
print()

Coord_spec = "O"
Bond_check = "H"
M_x, M_y, M_z = get_x_M(Bond_check)
spec_x, spec_y, spec_z  = get_x_M(Coord_spec)
n_M = np.size(M_x)
n_X = np.size(spec_x)
nOH_bonds = 0 
nH_Bonds = 0
nStrong = 0 
nMed = 0
nWeak = 0
for i in range(0, n_M):
    Strong_H_Bond_array = np.array([])
    Medium_H_Bond_array = np.array([])
    Weak_H_Bond_array = np.array([])
    for j in range(0, n_X):
        ABx, ABy, ABz = differences(spec_x[j], spec_y[j], spec_z[j], M_x[i], M_y[i], M_z[i]) 
        if Coord_system == "Hexagonal":
            ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        elif Coord_system != "Hexagonal":
            ABx, ABy, ABz = PBC_orthogonal(ABx, ABy, ABz)
            Bond_lenght = get_bond_lenght(ABx, ABy, ABz)
        if Bond_lenght < 1.2:
            nOH_bonds = nOH_bonds + 1
            for k in range(0, n_X):
                ABx, ABy, ABz = differences(spec_x[j], spec_y[j], spec_z[j], M_x[i], M_y[i], M_z[i]) 
                CBx, CBy, CBz = differences(spec_x[k], spec_y[k], spec_z[k], M_x[i], M_y[i], M_z[i]) 
 
                if Coord_system != "Hexagonal":   
                    ABx, ABy, ABz = PBC_orthogonal(ABx, ABy, ABz)
                    CBx, CBy, CBz = PBC_orthogonal(CBx, CBy, CBz)   

                elif Coord_system == "Hexagonal":
                    ABx, ABy, ABz = PBC_hexagonal(ABx, ABy, ABz)
                    CBx, CBy, CBz = PBC_hexagonal(CBx, CBy, CBz) 
                Bond_lenght_2 = get_bond_lenght(CBx, CBy, CBz)
                X_diff = (ABx * CBx)
                Y_diff = (ABy * CBy)
                Z_diff = (ABz * CBz)
        
                OHOangle = get_angle(X_diff, Y_diff, Z_diff, Bond_lenght, Bond_lenght_2)
                                
                if OHOangle > 90 and Bond_lenght_2 < 1.6 and Bond_lenght_2 > 1.2:
                    print("H -", i+1, "-", "Strong H-Bond -", (np.around(Bond_lenght_2, decimals=3)))
                    nH_Bonds = nH_Bonds + 1
                    nStrong = nStrong + 1
                    Strong_H_Bond_array = np.append(Strong_H_Bond_array, Bond_lenght_2)
                elif OHOangle > 90 and Bond_lenght_2 < 2.2 and Bond_lenght_2 > 1.6:
                    print("H -", i+1, "-", "Medium H-Bond -", (np.around(Bond_lenght_2, decimals=3)))
                    nH_Bonds = nH_Bonds + 1
                    nMed = nMed + 1
                    Medium_H_Bond_array = np.append(Medium_H_Bond_array, Bond_lenght_2)
                elif OHOangle > 90 and Bond_lenght_2 < 3.0 and Bond_lenght_2 > 2.2:
                    print("H -", i+1 , "-", "Weak H-bond -", (np.around(Bond_lenght_2, decimals=3)))
                    nH_Bonds = nH_Bonds + 1
                    nWeak = nWeak + 1
                    Weak_H_Bond_array = np.append(Weak_H_Bond_array, (np.around(Bond_lenght_2, decimals=3)))

print()
print("Total number of Hydrogen Bonds", nH_Bonds) 
print("Total number of Strong Hydrogen Bonds = ", nStrong)
print("Total number of Medium Hydrogen Bonds = ", nMed)
print("Total number of Weak Hydrogen Bonds = ", nWeak)

f.close()

Enter the atom you want to calculate bond lenghts for: Gd
Enter the coordinating species: O
Enter the Maximum bond lenght to be considered: 3

##############################################################
             Bond Lenght Information for Gd - O
##############################################################


Gd - 1 [ 2.361  2.55   2.512  2.312  2.319  2.222  2.379]

Average Bond Lenght for  Gd - 1 2.379
Range in Bond Lenght: 2.55  -  2.222


Coordination Number -  7


Gd - 2 [ 2.35   2.387  2.226  2.381  2.394  2.273]

Average Bond Lenght for  Gd - 2 2.335
Range in Bond Lenght: 2.394  -  2.226


Coordination Number -  6


Gd - 3 [ 2.398  2.383  2.234  2.356  2.356  2.265]

Average Bond Lenght for  Gd - 3 2.332
Range in Bond Lenght: 2.398  -  2.234


Coordination Number -  6


Gd - 4 [ 2.194  2.335  2.277  2.399  2.33   2.432]

Average Bond Lenght for  Gd - 4 2.328
Range in Bond Lenght: 2.432  -  2.194


Coordination Number -  6


Average Gd - O Bond Lenght:  2.345
Range in Bon